## Data Cleaning Outline:
***Documentation for our data cleaning process, including decisions regarding how we handle missing values, outliers, and other data quality issues.***



First, we import the necessary libraries and set the dataset which is a .csv file provided by the UVA School of Data Science and the UVA Department of Kinesiology as a pandas dataframe.

In [1]:
# Setting up our environment, importing all necessary libraries:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Importing dataset as dataframe:
df = pd.read_csv('aclr data(in).csv')

In [2]:
# Previewing the dataframe:
df.head()

,record_id,redcap_event_name,redcap_repeat_instrument,sex_dashboard,graft_dashboard2,med_meniscus,lat_meniscus,lat_stab,physis,visit_sex,...,lsi_flex_mvic_60,acl_ext_isok_60,con_ext_isok_60,lsi_ext_isok_60,acl_flex_isok_60,con_flex_isok_60,lsi_flex_isok_60,strength_testing_complete,rts,rts_tss
0,1,baseline_arm_1,NaN,Male,Other,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,visit_1_arm_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Male,...,NaN,2.57,2.92,87.86,1.50,1.45,103.32,2.0,NaN,NaN
2,1,long_term_outcomes_arm_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,70.0
3,2,baseline_arm_1,NaN,Female,HS autograft,1.0,3.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2,visit_1_arm_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Female,...,NaN,0.97,2.16,45.00,0.88,1.20,72.98,2.0,NaN,NaN


In [3]:
# Checking the dimensions of the dataframe:
print(df.shape)

(11150, 63)


The original dataframe has 11150 observations and 63 columns. We will be focusing on the variables we feel are most relevant to our hypothesis. We will be using the columns: `sex_dashboard`, `graft_dashboard2`, `reinjury`, `age`, `height_m`, `mass_kg`, `bmi`, `ikdc`, `acl_rsi` and dropping the rest from the dataframe.

In [4]:
df = df[['sex_dashboard', 'graft_dashboard2', 'reinjury', 'age', 'height_m', 'mass_kg', 'bmi', 'ikdc', 'acl_rsi', 'tss_dashboard']]
df.head()

,sex_dashboard,graft_dashboard2,reinjury,age,height_m,mass_kg,bmi,ikdc,acl_rsi,tss_dashboard
0,Male,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,No,21.7,1.9,87.4,24.210526,95.4,87.5,13 to 24 months
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Female,HS autograft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,No,14.5,1.6,72.2,28.203125,79.3,8.3,5 to 7 months


\
Now that we have our columns of interest, we will first check for missing values across the dataset. We will use the `isnull()` method to check for missing values and the `sum()` method to get the total number of missing values in each column, as well as the percentage of missing values in each column. 

In [5]:
# Checking for missing values:
missing_values = df.isnull().sum()

# Checking the percentage of missing values:
missing_percentage = (missing_values / len(df)) * 100
# Displaying missing values and their percentage:
missing_values = pd.DataFrame({'Missing Values': missing_values, 'Percentage': missing_percentage})

# Displaying the missing values:
print(missing_values)

                  Missing Values  Percentage
sex_dashboard               6413   57.515695
graft_dashboard2            6413   57.515695
reinjury                    5975   53.587444
age                         6024   54.026906
height_m                    8632   77.417040
mass_kg                     7899   70.843049
bmi                         8633   77.426009
ikdc                        8199   73.533632
acl_rsi                     7750   69.506726
tss_dashboard               5913   53.031390


\
Now we will proceed by separating the variables into categorical and continuous variables. We will use the `select_dtypes()` method to select the categorical variables and the continuous variables. For our numerical variables, we will impute missing values with the respective mean for each column.

In [20]:
# Filtering for numeric columns:
numeric_columns = df.select_dtypes(include=['int', 'float']).columns

# Imputing missing values with the mean for each respective column/varibale:
mean_values = df[numeric_columns].mean()
m_df = df.fillna(mean_values)

# Displaying the first 5 rows of the modified dataframe:
(m_df.head(5))

,sex_dashboard,graft_dashboard2,reinjury,age,height_m,mass_kg,bmi,ikdc,acl_rsi,tss_dashboard
0,Male,Other,NaN,20.184761,1.725412,74.343033,25.201579,78.457377,64.929381,NaN
1,NaN,NaN,No,21.700000,1.900000,87.400000,24.210526,95.400000,87.500000,13 to 24 months
2,NaN,NaN,NaN,20.184761,1.725412,74.343033,25.201579,78.457377,64.929381,NaN
3,Female,HS autograft,NaN,20.184761,1.725412,74.343033,25.201579,78.457377,64.929381,NaN
4,NaN,NaN,No,14.500000,1.600000,72.200000,28.203125,79.300000,8.300000,5 to 7 months


\
For our categorical variables, we have decided to fill the missing values with just an `Unknown` category, since this allows us to keep the rows with missing values without losing too much information so that we can continue with plotting later on.

In [22]:
# Filtering for Categorical columns:
categorical_columns = df.select_dtypes(include=['object']).columns
# Imputing missing values with the value 'Unknown' for each respective column/variable:
for column in categorical_columns:
    m_df[column] = m_df[column].fillna('Unknown')

# Displaying the first 5 rows of the modified dataframe:
(m_df.head(5))

,sex_dashboard,graft_dashboard2,reinjury,age,height_m,mass_kg,bmi,ikdc,acl_rsi,tss_dashboard
0,Male,Other,Unknown,20.184761,1.725412,74.343033,25.201579,78.457377,64.929381,Unknown
1,Unknown,Unknown,No,21.700000,1.900000,87.400000,24.210526,95.400000,87.500000,13 to 24 months
2,Unknown,Unknown,Unknown,20.184761,1.725412,74.343033,25.201579,78.457377,64.929381,Unknown
3,Female,HS autograft,Unknown,20.184761,1.725412,74.343033,25.201579,78.457377,64.929381,Unknown
4,Unknown,Unknown,No,14.500000,1.600000,72.200000,28.203125,79.300000,8.300000,5 to 7 months


\
Now we have finished our early data cleaning process and are ready to explore relations in our EDA process.